In [ ]:
!pip install gradio
!pip install diffusers torch
!pip install diffusers --upgrade
!pip install invisible_watermark transformers accelerate safetensors
!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 39.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 9.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 305.1/305.1 kB 28.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 381.9/381.9 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.3/60.3 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.0/70.0 kB 3.6 MB/s e

In [ ]:
import gradio as gr
import urllib.request
from bs4 import BeautifulSoup
from transformers import pipeline, BartTokenizer, BartForConditionalGeneration
import os
from diffusers.pipelines import DiffusionPipeline
from PIL import Image
import torch

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def dele():
    folder_path = '/content/drive/MyDrive/prompts/'

    # List all files in the folder
    files = os.listdir(folder_path)
    print(files)

    # Iterate through each file and attempt to delete it
    for file in files:
        file_path = os.path.join(folder_path, file)
        if os.path.isfile(file_path):
            os.remove(file_path)
            print(f"Deleted file: {file_path}")
        else:
            print(f"Skipping {file_path} as it is not a file.")

    print("All files deleted from the folder.")

# dele()


In [ ]:
def call2model(text):
    torch.cuda.empty_cache()
    prompts = []
    for sentence in text.split(". "):
        prompts.append(sentence)

    file_path = f"/content/drive/MyDrive/prompts/txtfile.txt"

    with open(file_path, 'w') as file:
        for prompt in prompts:
            file.write(prompt + "\n")

    pipe = DiffusionPipeline.from_pretrained("stabilityai/stable-diffusion-xl-base-1.0", torch_dtype=torch.float16, use_safetensors=True, variant="fp16")
    pipe.to("cuda")

    num_images_per_prompt = 5  # Adjust as needed

    print(len(prompts))
    for i in range(len(prompts)):
        for j in range(num_images_per_prompt):
            # Generate an image for each iteration
            images = pipe(prompt=prompts[i]).images[0]

            # Save each image with a unique filename
            image_filename = f"/content/drive/MyDrive/prompts/generated_image_{i}_{j}.png"

            images.save(image_filename)
    torch.cuda.empty_cache()

    %run '/content/drive/MyDrive/ColabNotebooks/Meghana-Adding_audio_n_combine.ipynb'

    torch.cuda.empty_cache()

In [ ]:
# Define the function to find the main content
def find_main_content(soup):
    potential_tags = ['article', 'main', 'div', 'section']

    for tag in potential_tags:
        main_content = soup.find(tag)
        if main_content:
            return main_content

    return None

# Define the function to process the URL and generate a summary
def process_url(url):
    # Delete the ild stuff in the folder
    dele()
    # Fetch the web page content
    source = urllib.request.urlopen(url).read()
    soup = BeautifulSoup(source, 'html.parser')

    # Find the main content automatically
    main_content = find_main_content(soup)

    # Check if main_content is not None before extracting paragraphs
    if main_content:
        # Extract and concatenate text from paragraphs within the main content
        text = ""
        for paragraph in main_content.find_all('p'):
            text += paragraph.text

        # Load BART model and tokenizer
        tokenizer = BartTokenizer.from_pretrained("facebook/bart-large-cnn")
        model = BartForConditionalGeneration.from_pretrained("facebook/bart-large-cnn")

        # Tokenize and generate summary
        inputs = tokenizer(text, return_tensors="pt", max_length=1024, truncation=True)
        summary_ids = model.generate(inputs["input_ids"], max_length=250, min_length=100, length_penalty=2.0, num_beams=4, temperature=0.7)

        # Decode and return the generated summary
        summary_text = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
        print(summary_text)
        #Call diffusion model
        call2model(summary_text)
        return "/content/drive/MyDrive/prompts/output_concatenated_video_new.mp4"
        # return {"message": "Video created and saved to Google Drive."}

    else:
        return {"Some error occured!!"}



In [ ]:
def combine(url):
  # torch.cuda.empty_cache()
  %run '/content/drive/MyDrive/ColabNotebooks/Meghana-Adding_audio_n_combine.ipynb'
  return "/content/drive/MyDrive/prompts/output_concatenated_video_new.mp4"
  # torch.cuda.empty_cache()  if you have any gpu issues, this will clear the cache

In [ ]:
# Gradio interface setup
!pip install gradio
import gradio as gr
iface = gr.Interface(
    # fn=process_url,
    fn=combine,
    inputs=gr.Textbox(placeholder="Enter the news article link"),
    outputs=gr.Video(),
    title="Press Release Video",
    description="Enter a news article link, and the model will generate a video."
)

iface.launch(debug=True)
